In [ ]:
import joblib
import pandas as pd
import re
import numpy as np
import nltk as tk
import pickle

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn.model_selection import train_test_split

tk.download('stopwords')
tk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
suicide_dt = pd.read_csv(r'/content/drive/MyDrive/Suicide_Detection.csv')

In [ ]:
#suicide_dt.dropna(inplace=True)

In [ ]:
suicide_dt.head(5)

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
suicide_dt['text'].value_counts() #count the number of rows for each class

Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.                                                                                                                                                                                                                                                                                             

In [ ]:
def text_cleaning(text):
    list_subreddit = []
    for reddit in tqdm(text):
        reddit = reddit.lower()
        reddit = re.sub(r'\s+[a-z]\s+', ' ',reddit) #removing single characters
        reddit = re.sub(r'^[a-z]\s+',' ', reddit) #removing single characters at the start of sentece I & space
        reddit = re.sub(r'\W',' ', reddit) # Removing special characters
        reddit = re.sub(r'\d',' ', reddit) #Removing digits
        reddit = re.sub(r'^\s+[a-z]$',' ', reddit)
        reddit = re.sub(r'https://t.co/[a-zA-Z0-9]*\s', ' ', reddit)
        reddit = re.sub(r'\s+https://t.co/[a-zA-Z0-9]*\s', ' ', reddit)
        reddit = re.sub(r'\s+https://t.co/[a-zA-Z0-9]*$', ' ', reddit)
        reddit  = re.sub(r"won't" ,"will not" , reddit)
        reddit  = re.sub(r"that's" ,"that is" , reddit)
        reddit = re.sub(r"what's","what is", reddit)
        reddit = re.sub(r"it's","it is", reddit)
        reddit = re.sub(r"who's","who is", reddit)
        reddit = re.sub(r"she's","she is", reddit)
        reddit = re.sub(r"i'm","i am", reddit)
        reddit = re.sub(r"he's","he is", reddit)
        reddit = re.sub(r"they're","they are", reddit)
        reddit = re.sub(r"ain't","am not", reddit)
        reddit = re.sub(r"wouldn't","would not", reddit)
        reddit = re.sub(r"shouldn't","should not", reddit)
        reddit = re.sub(r"can't","can not", reddit)
        reddit = re.sub(r"couldn't","could not", reddit)
        reddit = re.sub(r"won't","would not", reddit)

        
        list_subreddit.append(reddit)

    return list_subreddit

suicide_dt["text"]= text_cleaning(suicide_dt["text"])

100%|██████████| 232074/232074 [01:41<00:00, 2297.30it/s]


In [ ]:
words = stopwords.words("english")

In [ ]:
tk.download('wordnet')
wl = WordNetLemmatizer()
suicide_dt['text'] = suicide_dt['text'].apply(
    lambda x: " ".join([wl.lemmatize(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
suicide_dt.head(5)

,Unnamed: 0,text,class
0,2,ex wife threatening suiciderecently left wife ...,suicide
1,3,weird get affected compliment coming someone k...,non-suicide
2,4,finally almost never hear bad year ever swear ...,non-suicide
3,8,need helpjust help im cry hard,suicide
4,9,losthello name adam struggling year afraid pas...,suicide


In [ ]:
#Label Encoding refers to converting the labels into a numeric form 
#https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

suicide_dt['target_variable']= label_encoder.fit_transform(suicide_dt['class'])

suicide_dt.head()

,Unnamed: 0,text,class,target_variable
0,2,ex wife threatening suiciderecently left wife ...,suicide,1
1,3,weird get affected compliment coming someone k...,non-suicide,0
2,4,finally almost never hear bad year ever swear ...,non-suicide,0
3,8,need helpjust help im cry hard,suicide,1
4,9,losthello name adam struggling year afraid pas...,suicide,1


In [ ]:
#using pickle to load our dataset faster
with open('suicide_pickle.pickle', 'wb') as f: # we are creating pickle files for Suicide_dt, wb is for bytes
    pickle.dump(suicide_dt,f)
    
with open('suicide_pickle.pickle', 'rb') as f:
    p_suicide_dt = pickle.load(f)

In [ ]:
#Spliting document and target variable into different variables
X = p_suicide_dt['text']
Y = p_suicide_dt['target_variable']

In [ ]:
#text_train is to train our data, text_test is our test data
#sent_train is our classes associated with text_train, similar for sent_test
text_train, text_test, sent_train, sent_test = train_test_split(X,Y,test_size=0.33,random_state=0)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
pipe = Pipeline([('tfidf',TfidfVectorizer(ngram_range=(1,2),stop_words="english", sublinear_tf=True)),
                 ('chi', SelectKBest(chi2, k=10000)),
                 ('clf', LogisticRegression(max_iter=300, class_weight='balanced'))
]) 

In [ ]:
from sklearn.pipeline import Pipeline
model = pipe.fit(text_train, sent_train)

In [ ]:
print("Accuracy Score: " + str(model.score(text_test, sent_test)))

Accuracy Score: 0.9299993471306391


In [ ]:
ml_model = "ml_model.pkl"
joblib.dump(model, ml_model)

['ml_model.pkl']